# Минимизация квадратичной функции

# Метод наискорейшего градиентного спуска (МНГС) 

### импорт библиотек

In [1]:
import sympy as sym
import numpy as np

In [2]:
# функция транспонирования матрицы
def transpose(A):
    n = len(A)
    A1 = np.copy(A)
    for i in range(n):
        for j in range(n):
            A1[i][j] = A[j][i]
    return A1
def vec_transpose(q):
    n = len(q)
    ans = []


### ввод данных

In [3]:
x, y, z = sym.symbols('x y z')  # ввели символьные переменные
f = sym.Function('f')(x, y, z)  # ввели символьную функцию
N = 1  # номер в списке группы
f = 2*x**2+(3+0.1*N)*y**2+(4+0.1*N)*z**2+x*y-y*z+x*z+x-2*y+3*z+N  # заданная для минимизации функция
# f = 2x^2+3.1y^2+4.1z^2+xy-yz+xz+x-2y+3z+1
eps = 10**(-6)  # заданная погрешность измерений

# f представляется в виде f = 1/2*xT*A*X + xT*b + N
A = [[4, 1, 1], [1, 6.2, -1], [1, -1, 8.2]]  
b = [1, -2, 3]

# gradf = sym.Function('gradf')(x, y, z)  # градиент функции f
gradf = [4*x+y+z+1, x+6.2*y-z-2, x-y+8.2*z+3]  # посчитан вручную, 
    # либо с помощью встроенной функции gradf = [sym.diff(f,x), sym.diff(f, y), sym.diff(f,z)]
    # либо учитывая что для квадратичной функции градиент имеет вид gradf = Ax+b
x0 = [0, 0, 0]  # начальное приближение, (метод сходится для любого начального х0)
xk = [10, 10, 10]  # проинициализировано только для захода в цикл

# Воспользовавшись формулой (2.6) в методичке, можно оценить расстояние от произвольной точки
# до точки минимума

eig = np.linalg.eigvals(A)  # собственные числа матрицы А
print("собственные числа: ", eig)
m=min(eig) # минимальное собственное число
print("m =", m)

собственные числа:  [3.26737156 6.44692569 8.68570275]
m = 3.267371563474186


### определение дополнительных функций для вычислений

In [4]:
# евклидова норма вектора 
def norm2vec(x): 
    x = [a**2 for a in x ] # возьмем каждый элемент вектора в квадрате
    return sum(x)**(1/2)
# умножение вектора на число
def vec_mul_number(x, n):
    return [a*n for a in x]

### Алгоритм

In [9]:
q = np.array([gradf[0].subs([(x, x0[0]), (y, x0[1]),(z, x0[2])]), 
              gradf[1].subs([(x, x0[0]), (y, x0[1]),(z, x0[2])]),
              gradf[2].subs([(x, x0[0]), (y, x0[1]),(z, x0[2])])])

qnorm = norm2vec(q)
print(qnorm)
qt = transpose(q)  # q транспонированное
mu = -qnorm/((qt.dot(A).dot(q)))  # мю
xk = x0 + np.array(vec_mul_number(q, mu)) #xk+1
while (norm2vec(q)/m >= eps):
    x0=xk  # x_k
    q = np.array([gradf[0].subs([(x, x0[0]), (y, x0[1]),(z, x0[2])]), 
                  gradf[1].subs([(x, x0[0]), (y, x0[1]),(z, x0[2])]),
                  gradf[2].subs([(x, x0[0]), (y, x0[1]),(z, x0[2])])])
    qnorm = norm2vec(q)
    qt = transpose(q)
    mu = -qnorm**2/((qt.dot(A).dot(q)))
    xk = x0 + vec_mul_number(q, mu)  # x_k+1
    print([a.round(6) for a in xk], [a.round(6) for a in q], mu.round(6))
print("xk = ", xk)

3.74165738677394


TypeError: 'One' object is not subscriptable

### Результаты

In [5]:
f_xk = f.subs([(x, xk[0]), (y, xk[1]),(z, xk[2])])
print("f(xk) = ", f_xk)
fmin = 0.11226497595  # точное значение функции в точке минимума, посчитанное при помощи Wolfram Alpha
print("Delta = ", abs(fmin-f_xk))  # Delta - абсолютная погрешность

f(xk) =  0.112264975951105
Delta =  1.10540743225584e-12


# Метод наискорейшего покоординатного спуска

### ввод данных
аналогичен, добавлен для наглядности в дальнейшем

In [6]:
x, y, z = sym.symbols('x y z')
f = sym.Function('f')(x, y, z)
N = 1
f = 2*x**2+(3+0.1*N)*y**2+(4+0.1*N)*z**2+x*y-y*z+x*z+x-2*y+3*z+N
eps = 10**(-6)
A = np.array([[4, 1, 1], [1, 6.2, -1], [1, -1, 8.2]])
b = [1, -2, 3]
gradf=sym.Function('gradf')(x, y, z)
gradf = [4*x+y+z+1, x+6.2*y-z-2, x-y+8.2*z+3]
x0 = np.array([10, 10, 10]) 
xk = np.zeros(3)
eig = np.linalg.eigvals(A)
m=min(eig)

### Алгоритм

In [7]:
while (norm2vec(A.dot(xk)+b)/m >= eps):
    for i in range(3):
        x0=xk
        q = np.array(np.eye(3)[i])  # q = e_i, где e_i - единичный орт 3х3
        qt = q.transpose()
        mu = -q.dot(A.dot(x0)+b)/((qt.dot(A).dot(q)))
        xk = x0 + vec_mul_number(q, mu) #xk+1
        print([a.round(6) for a in xk], [a.round(6) for a in q], mu.round(6))
print("xk = ", xk)

[-0.25, 0.0, 0.0] [1.0, 0.0, 0.0] -0.25
[-0.25, 0.362903, 0.0] [0.0, 1.0, 0.0] 0.362903
[-0.25, 0.362903, -0.291109] [0.0, 0.0, 1.0] -0.291109
[-0.267948, 0.362903, -0.291109] [1.0, 0.0, 0.0] -0.017948
[-0.267948, 0.318845, -0.291109] [0.0, 1.0, 0.0] -0.044058
[-0.267948, 0.318845, -0.294293] [0.0, 0.0, 1.0] -0.003184
[-0.256138, 0.318845, -0.294293] [1.0, 0.0, 0.0] 0.011811
[-0.256138, 0.316427, -0.294293] [0.0, 1.0, 0.0] -0.002418
[-0.256138, 0.316427, -0.296029] [0.0, 0.0, 1.0] -0.001735
[-0.255099, 0.316427, -0.296029] [1.0, 0.0, 0.0] 0.001038
[-0.255099, 0.315979, -0.296029] [0.0, 1.0, 0.0] -0.000447
[-0.255099, 0.315979, -0.29621] [0.0, 0.0, 1.0] -0.000181
[-0.254942, 0.315979, -0.29621] [1.0, 0.0, 0.0] 0.000157
[-0.254942, 0.315925, -0.29621] [0.0, 1.0, 0.0] -5.5e-05
[-0.254942, 0.315925, -0.296236] [0.0, 0.0, 1.0] -2.6e-05
[-0.254922, 0.315925, -0.296236] [1.0, 0.0, 0.0] 2e-05
[-0.254922, 0.315917, -0.296236] [0.0, 1.0, 0.0] -7e-06
[-0.254922, 0.315917, -0.296239] [0.0, 0.0, 1.

### Результаты

In [8]:
f_xk = f.subs([(x, xk[0]), (y, xk[1]),(z, xk[2])])
print("f(xk) = ", f_xk)
fmin = 0.11226497595  # точное значение функции в точке минимума, посчитанное при помощи Wolfram Alpha
print("Delta = ", abs(fmin-f_xk))

f(xk) =  0.112264975951352
Delta =  1.35209898832755e-12
